# GPT-2 학습해보기

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

이번 시간엔 한국어 코퍼스를 활용해, 직접 한국어 GPT-2를 학습해보겠습니다.

In [4]:
!pip install transformers

     |████████████████████████████████| 2.2MB 6.5MB/s 
     |████████████████████████████████| 870kB 26.4MB/s 
     |████████████████████████████████| 3.3MB 33.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=a4ef502623620acf04f19d8fdeae7015039cb6ede1618c4a4765d4ff30c46c7c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [5]:
import torch
torch.cuda.is_available()

True

역시 위키 데이터를 가져와볼까요?

In [1]:
!mkdir my_data
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    532      0 --:--:-- --:--:-- --:--:--   531
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  1284k      0  0:00:01  0:00:01 --:--:--  167M


In [2]:
path = "/content/my_data/wiki_20190620_small.txt"

지금까지는 BertWordPieceTokenizer를 사용해왔다면,   
이번에는 SentencePiceBPETokenizer를 사용해 모델을 학습해보겠습니다.

각 tokenizer의 차이는 허훈님의 블로그 [여기](https://huffon.github.io/2020/07/05/tokenizers/) 에서 확인하실 수 있습니다.



In [6]:
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer

tokenizer = SentencePieceBPETokenizer()

tokenizer._tokenizer.normalizer = BertNormalizer(clean_text=True,
handle_chinese_chars=False,
lowercase=False)

tokenizer.train(
    path,
    vocab_size=10000,
    special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
    ],
)



In [7]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))
# SentencePiece를 사용하면, 나중에 decoding 과정에서 '_' 만 ' '로 replace해주면 띄어쓰기 복원이 가능해집니다.

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[1005, 577, 6613, 1303, 1041, 2071, 1136, 594, 1033]
['▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.']
이순신은 조선 중기의 무신이다.


In [ ]:
tokenizer.save_model(".")

['./vocab.json', './merges.txt']

In [ ]:
tokenizer = SentencePieceBPETokenizer.from_file(vocab_filename="vocab.json", merges_filename="merges.txt")

In [ ]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[1005, 578, 6613, 1303, 1041, 2071, 1136, 595, 1033]
['▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.']
['▁<', 's', '>', '이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '<', '/s', '>']
<s>이순신은 조선 중기의 무신이다.</s>


In [ ]:
tokenizer.add_special_tokens(["<s>", "</s>", "<unk>", "<pad>", "<shkim>"])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<bos>")
tokenizer.eos_token_id = tokenizer.token_to_id("<eos>")

print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

[0, 1005, 578, 6613, 1303, 1041, 2071, 1136, 595, 1033, 2]
['<s>', '▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '</s>']
이순신은 조선 중기의 무신이다.


In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.get_vocab_size(),
  bos_token_id=tokenizer.token_to_id("<s>"),
  eos_token_id=tokenizer.token_to_id("</s>"),
)
# creating the model
model = GPT2LMHeadModel(config)

In [ ]:
model.num_parameters()

93522432

In [ ]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 데이터셋을 만들기 시작합니다.
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f:
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가해줍니다.
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합쳐버립니다 :-)
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length만큼 데이터를 잘라서 저장합니다.
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [ ]:
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
print(dataset[0])

tensor([   0, 3997, 3546, 8404,  464,    4, 5480, 9527, 1798, 1890, 2297, 1262,
        9626, 2679, 1188, 2174,    2,    0, 5708, 5480,  255, 6466,  750, 3426,
         872, 1556,  680,  895, 1626, 9223,  587, 3621, 1010, 3303,    2,    0,
        6466, 7418, 2305,  404, 2217, 1074,    2,    0, 1013, 1107, 3716,  646,
        8574, 1024,  940,   92, 7323,  372,   92,  721, 9295,  705, 1651,  454,
        3166, 1032, 1074,    2,    0, 6343, 1262, 3716, 1009, 2932, 1176,  913,
        2036, 1171, 3227,  843,   92,  440,  974, 1486, 1017,    3, 1323, 3914,
        2095, 1042,    2,    0, 1383, 2068, 2225, 1095,  327,  843, 1824,  507,
           4, 1240, 7698,    2,    0, 3897, 6466, 1053, 1077,  686, 2318, 4649,
        5204, 5671, 1013, 1759,  115, 2742, 3004,  104,  655, 2283, 9765, 1192,
        1796, 2449, 2546, 9939, 6466, 1053, 1037,  534])


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=64, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)


In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
USE_GPU = 1
# Device configuration
device = torch.device('cuda' if (torch.cuda.is_available() and USE_GPU) else 'cpu')

In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("<s>이순신", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


GENERATED SEQUENCE : 이순신의 반은 인으로 데인 반라도 가장 많이 오자가 되었다.
GENERATED SEQUENCE : 이순신적인 것은 '사가 큰 도문에 비해 만들어 것을 하는 것이다.
GENERATED SEQUENCE : 이순신가 지 두 원을 시작은 그의 정아야체의 같은 리의 때문에 191인 때 전로 사용하여, 그는 "용카키레한 후키에 의해 위해 “트를 "카라 가져, 로예트를 고통되어 중국 비디오 대전할 수 없는 「스 바린 시작되어 여러 차례 5월인 독일·자한 자전적인 소설라키의 중국계리문이다.
